In [88]:
import pandas as pd
from futils import ROOT
from futils import WrapPath
full_df_filtered = pd.read_feather(ROOT / "data/gnina_processed/LIT-PCBA/filtered.feather")
n_df = pd.read_csv(ROOT / "data/raw_data/LIT-PCBA/AVE_unbiased/n.csv", index_col = 0)
from os.path import join
import rdkit.Chem as Chem
SDF = Chem.SDMolSupplier
import os
import polars as pl
from polars import col
from evaluate_lit_pcba import get_subset, get_data
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import imgkit

In [4]:
# full_df_filtered_affinity = full_df.sort_values(sort_according_to, ascending = False).drop_duplicates(["_Name", "directory", "ligands"])
# full_df_filtered_affinity["active_groundtruth"] = full_df_filtered_affinity["ligands"].str.contains("_active_")
# full_df_filtered.reset_index().rename(columns = {"level_0": "unfiltered_index"}).to_feather(join(inputdir, "filtered.feather"))

In [73]:
giant_gnina = pd.read_parquet("/home/qzj517/POR-DD/gnina_extract_affinities/df.parquet")
giant_gnina = giant_gnina.rename(columns = {"activity": "active_groundtruth", "target": "directory"})
giant_gnina["active_groundtruth"] = giant_gnina["active_groundtruth"] == "active"
giant_gnina = giant_gnina.sort_values("CNNaffinity_times_score", ascending = False)
not_unbiased_n_df = pd.read_csv("/home/qzj517/POR-DD/data/raw_data/LIT-PCBA/full_data/n.csv", index_col = 0)
not_unbiased_n_df.loc["ADRB2"] = n_df.loc["ADRB2"]
pass

In [121]:
gnina_VS = get_data(giant_gnina, not_unbiased_n_df)
gnina_VS.index.name = "target"
gnina_VS.loc["mean", ["EF1", "NEF1"]] = gnina_VS[["EF1", "NEF1"]].mean().to_list()
gnina_VS.loc["median", ["EF1", "NEF1"]] = gnina_VS[["EF1", "NEF1"]].median().to_list()


In [117]:
VS = get_data(full_df_filtered, n_df).sort_index()
VS.index.name = "target"
VS.loc["mean", ["EF1", "NEF1"]] = VS[["EF1", "NEF1"]].mean().to_list()
VS.loc["median", ["EF1", "NEF1"]] = VS[["EF1", "NEF1"]].median().to_list()

In [142]:
VS[["EF1", "NEF1"]].style.set_caption("Equibind+gnina").format("{:.4g}")


,EF1,NEF1
target,,
ADRB2,11.77,0.1176
ALDH1,1.321,0.06526
ESR1_ago,7.687,0.07692
ESR1_ant,10.68,0.2157
FEN1,2.44,0.02439
GBA,3.617,0.03614
IDH1,2.565,0.02564
KAT2A,1.547,0.01546
MAPK1,2.924,0.02922


In [141]:
gnina_VS[["EF1", "NEF1"]].style.set_caption("Full gnina docking").format("{:.4g}")

,EF1,NEF1
target,,
ADRB2,5.882,0.05882
ALDH1,1.758,0.08684
ESR1_ago,7.687,0.07692
ESR1_ant,7.766,0.1569
FEN1,2.44,0.02439
GBA,8.437,0.08434
IDH1,5.128,0.05128
KAT2A,0,0
MAPK1,2.274,0.02273


In [116]:
gnina_VS

,EF1,NEF1,n_actives_found,1% is equal to,gt_active_proportion,total_succeeded,total_ligands
target,,,,,,,
ADRB2,5.882353,0.058824,1.0,3125.0,0.000054,312436.0,312500.0
ALDH1,1.758212,0.086837,126.0,1451.0,0.049389,145026.0,145133.0
ESR1_ago,7.686813,0.076923,1.0,56.0,0.002323,5595.0,5596.0
ESR1_ant,7.766244,0.156863,8.0,51.0,0.020198,5049.0,5050.0
FEN1,2.439511,0.02439,9.0,3557.0,0.001037,355697.0,355771.0
GBA,8.437096,0.084337,14.0,2961.0,0.000560,296095.0,296218.0
IDH1,5.128035,0.051282,2.0,3621.0,0.000108,362015.0,362088.0
KAT2A,0.0,0.0,0.0,3487.0,0.000556,348626.0,348742.0
MAPK1,2.274064,0.022727,7.0,629.0,0.004894,62849.0,62937.0


In [42]:
curdir, folders, files = next(os.walk("/home/qzj517/POR-DD/data/raw_data/LIT-PCBA/full_data"))
data = []
for folder in folders:
    with open(Path(curdir) / folder / "actives.smi") as file:
        actives = sum(1 for line in file)
    with open(Path(curdir) / folder / "inactives.smi") as file:
        inactives = sum(1 for line in file)
    data.append({"target": folder, "actives": actives, "total": actives + inactives})

In [47]:
pd.DataFrame(data).set_index("target").to_csv("/home/qzj517/POR-DD/data/raw_data/LIT-PCBA/full_data/n.csv")

In [38]:
n_df

,actives,total
TP53,79,4247
ADRB2,17,312500
KAT2A,194,348742
OPRK1,24,269840
MTORC1,97,33069
ESR1_ant,102,5050
FEN1,369,355771
PKM2,546,246069
VDR,663,267204
ALDH1,5376,108850


In [5]:
total_failed = VS.total_ligands - VS.total_succeeded

In [3]:
import importlib
import failed_ligands
importlib.reload(failed_ligands)
df = failed_ligands.get_failed_entries(ROOT / "data/equibind_processed/LIT-PCBA")
if failed_ligands.check_if_all_proteins_fail_on_identical_ligands(df):
    df = failed_ligands.get_first_protein(df)
else:
    print("PANIC: not all proteins fail on identical ligands")

In [7]:
df.apply(len).groupby(["target"]).sum()

target
ADRB2       181
ALDH1       150
ESR1_ago      1
ESR1_ant      1
FEN1        219
GBA         245
IDH1        216
KAT2A       256
MAPK1       122
MTORC1       26
OPRK1       172
PKM2        168
PPARG         1
TP53          1
VDR         202
Name: failed, dtype: int64

In [39]:
mols = failed_ligands.put_all_failed_into_one_file(df, ROOT / "data/raw_data/LIT-PCBA/AVE_unbiased", "")

In [41]:
supp = Chem.SmilesMolSupplier("failed_ligands/all_failed.smi")

In [49]:
from rdkit.Chem import SanitizeMol, SanitizeFlags, AddHs, AllChem
import torch
def _process(lig):
    addH = True
    generate_conformer = True
    if lig is None:
        return None, None
    if addH:
        lig = AddHs(lig)
    if generate_conformer:
        try:
            get_rdkit_coords(lig)
        except ValueError:
            return None, lig.GetProp("_Name")
    sanitize_succeded = (SanitizeMol(lig, catchErrors = True) is SanitizeFlags.SANITIZE_NONE)
    if sanitize_succeded:
        return lig, lig.GetProp("_Name")
    else:
        return None, lig.GetProp("_Name")

In [50]:
def get_rdkit_coords(mol, seed = None):
    ps = AllChem.ETKDGv2()
    if seed is not None:
        ps.randomSeed = seed
    id = AllChem.EmbedMolecule(mol, ps)
    if id == -1:
        print('rdkit coords could not be generated without using random coords. using random coords now.')
        ps.useRandomCoords = True
        AllChem.EmbedMolecule(mol, ps)
        AllChem.MMFFOptimizeMolecule(mol, confId=0)
    else:
        AllChem.MMFFOptimizeMolecule(mol, confId=0)
    conf = mol.GetConformer()
    lig_coords = conf.GetPositions()
    return torch.tensor(lig_coords, dtype=torch.float32)

In [102]:
def get_names(protname):
    dirname = "../../data/raw_data/LIT-PCBA/AVE_unbiased"
    smiles_files = [join(dirname, protname, file) for file in os.listdir(join(dirname, protname)) if file.endswith(".smi")]
    names = {}
    for smiles_file in smiles_files:
        names[smiles_file] = []
        sup = Chem.SmilesMolSupplier(smiles_file, titleLine=False)
        for mol in sup:
            names[smiles_file].append(mol.GetProp("_Name"))
    return names

In [103]:
data = get_names("OPRK1")